In [ ]:
# =============================================
# MÓDULO_07: Generación de Reporte Visual Final
# =============================================

# ---------------------------------------------
# 1. Configuración Inicial del Entorno
# ---------------------------------------------
nombre_lote = "LOTE_20250614"
nombre_modulo = "MODULO_07"

try:
    import google.colab
    EN_COLAB = True
except ImportError:
    EN_COLAB = False

if EN_COLAB:
    from google.colab import drive
    drive.mount("/content/drive", force_remount=True)
    ruta_base = "/content/drive/MyDrive/TFM_EVA_MARTIN/Modulos"
else:
    ruta_base = "G:/Mi unidad/TFM_EVA_MARTIN/Modulos"

import os
import sys
import logging

if ruta_base not in sys.path:
    sys.path.append(ruta_base)

import utilidades_comunes

# Cargar configuración YAML
import yaml
ruta_config = os.path.join(ruta_base, "config.yaml")
with open(ruta_config, "r", encoding="utf-8") as f:
    config = yaml.safe_load(f)
params = config["parametros"]

# Configurar entorno y logger
logger = utilidades_comunes.configurar_logger(nombre_modulo)
logger.setLevel(logging.DEBUG)
entorno = utilidades_comunes.inicializar_entorno(nombre_modulo, nombre_lote, ruta_base)

# ---------------------------------------------
# 2. Cargar métricas desde el módulo anterior
# ---------------------------------------------
import pandas as pd
ruta_csv = os.path.join(ruta_base, "MODULO_06", "salida", "evaluacion_metrica_detallada.csv")
report_df = pd.read_csv(ruta_csv, index_col=0)

logger.info(f"📊 Métricas cargadas: {report_df.shape[0]} filas")

# ---------------------------------------------
# 3. Visualizaciones
# ---------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Crear carpeta de salida de gráficos
carpeta_figs = os.path.join(entorno["ruta_salida"], "figuras")
os.makedirs(carpeta_figs, exist_ok=True)

# 3.1: Barras comparativas por clase
metricas = ["precision", "recall", "f1-score"]
report_df_clases = report_df.loc[["Positivo", "Neutro", "Negativo"]][metricas]

plt.figure(figsize=(8, 6))
report_df_clases.plot(kind="bar")
plt.title("Métricas por Clase")
plt.ylim(0, 1.05)
plt.ylabel("Valor")
plt.xlabel("Clase")
plt.xticks(rotation=0)
plt.legend(loc="lower right")
plt.tight_layout()
fig_path = os.path.join(carpeta_figs, "metricas_por_clase.png")
plt.savefig(fig_path)
plt.close()

logger.info(f"📈 Gráfico de métricas por clase guardado en: {fig_path}")

# 3.2: Matriz de Confusión (si tienes y_true, y_pred)
# Suponiendo que se guarda desde el módulo 06 como opcional
try:
    ruta_preds = os.path.join(ruta_base, "MODULO_06", "salida", "predicciones.csv")
    df_preds = pd.read_csv(ruta_preds)
    from sklearn.metrics import confusion_matrix
    import numpy as np

    y_true = df_preds["true"]
    y_pred = df_preds["pred"]

    labels = ["Positivo", "Neutro", "Negativo"]
    cm = confusion_matrix(y_true, y_pred, labels=range(3))

    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels)
    plt.title("Matriz de Confusión")
    plt.ylabel("Etiqueta Real")
    plt.xlabel("Predicción")
    plt.tight_layout()
    fig_cm_path = os.path.join(carpeta_figs, "matriz_confusion.png")
    plt.savefig(fig_cm_path)
    plt.close()
    logger.info(f"🔷 Matriz de confusión guardada en: {fig_cm_path}")
except Exception as e:
    logger.warning(f"No se pudo generar matriz de confusión: {e}")

logger.info(f"✅ Finalización del módulo {nombre_modulo} con visualizaciones generadas")
